In [139]:
!pip install psutil
 

DEPRECATION: Loading egg at d:\anaconda\lib\site-packages\huggingface_hub-0.19.4-py3.8.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [140]:
import time
import psutil
import json
import os
import zlib
import re
import tracemalloc
import math
import collections

# inverted  index

In [141]:
 
def create_inverted_index(file_path):
    inverted_index = {}
    
    with open (input_file, 'r', encoding='latin-1') as file:
        current_term = None
        current_docs = []
        
        for line in file:
            term, doc_id = line.strip().split()
            
            if term != current_term:
                if current_term is not None:
                    inverted_index[current_term] = current_docs
                current_term = term
                current_docs = [doc_id]
            else:
                current_docs.append(doc_id)
        
        # Adding the last term-doc pair
        if current_term is not None:
            inverted_index[current_term] = current_docs
    
    return inverted_index    

#  persist the inverted index

In [142]:
def save_inverted_index(inverted_index, output_file):
    with open(output_file, 'w', encoding='utf-8') as file:
        json.dump(inverted_index, file, ensure_ascii=False, indent=4)

def load_inverted_index(input_file):
    with open(input_file, 'r', encoding='utf-8') as file:
        inverted_index = json.load(file)
    return inverted_index

In [154]:
 
input_file = 'output/output.txt'  # Path to the input file
output_file = 'inverted_index1.json'  # Output file path
inverted_index = create_inverted_index(input_file)
save_inverted_index(inverted_index, output_file)


# Boolean Search with AND

In [155]:
def boolean_search_AND(query, inverted_index):

    query_terms = query.split()
    result = None
    
    # Ensure that all query terms exist in the inverted index
    for term in query_terms:
        if term not in inverted_index:
            return []
    # Initialize result with the documents containing the first term
    result = set(inverted_index[query_terms[0]])
    # Perform AND operation with other terms
    for term in query_terms[1:]:
        result = result.intersection(inverted_index[term])
        
    process = psutil.Process(os.getpid())
    memory_usage = process.memory_info().rss / (1024 * 1024)  # 转换为MB 
   

    return list(result),memory_usage 

In [156]:
 
def boolean_search_AND_encode1(encoded_dict, query):
    # 检查查询词是否都在压缩字典中
    query_terms = query.split()
    for term in query_terms:
        if term not in encoded_dict:
            return "Query term '{}' not found in dictionary".format(term)
    
    
    postings_lists = []
     
    for term in query_terms:
        encoded_postings = encoded_dict[term]
        compressed_postings = json.loads(zlib.decompress(encoded_postings).decode())
        
        postings = []
        prev_doc_id = 0
        for gap in compressed_postings:
            doc_id = prev_doc_id + gap
            postings.append(doc_id)
            prev_doc_id = doc_id
        
      
        postings_lists.append(set(postings))
    
    result = set.intersection(*postings_lists)
    
    process = psutil.Process(os.getpid())
    memory_usage = process.memory_info().rss / (1024 * 1024)  # 转换为MB         
    return list(result), memory_usage
  
 
    


# Boolean Search with ORandNOT

In [157]:
def boolean_search_ORandNOT(query, inverted_index):
    query_terms = query.split()
    result = set()
    # Ensure that all query terms exist in the inverted index
    for term in query_terms:
        if term not in inverted_index:
            return []
    # Check for presence of NOT terms
    not_terms = [term[1:] for term in query_terms if term.startswith('-')]
    # Initialize result with the documents containing the first non-NOT term
    for term in query_terms:
        if not term.startswith('-'):
            result.update(inverted_index[term])
    # Perform AND operation with other non-NOT terms
    for term in query_terms:
        if not term.startswith('-'):
            result = result.intersection(inverted_index[term])
    # Exclude documents containing the NOT terms
    for term in not_terms:
        if term in inverted_index:
            result.difference_update(inverted_index[term])
    process = psutil.Process(os.getpid())
    memory_usage = process.memory_info().rss / (1024 * 1024)  # 转换为MB         
    return list(result), memory_usage
 

# Index Compression/Optimisation

In [158]:


def extract_number_from_path(document_path):
    # 使用正则表达式匹配文档路径中的第一个数字
    match = re.search(r'\d+', document_path)
    if match:
        return int(match.group())
    else:
        return None

def compress_postings(postings):
    compressed_postings = []
    prev_doc_id = 0
    for doc_id in postings:
        doc_id_hash = extract_number_from_path(doc_id)
        gap = doc_id_hash - prev_doc_id   
        compressed_postings.append(gap)
        prev_doc_id = doc_id_hash  
    return compressed_postings

def encode_dictionary(dictionary):
    encoded_dict = {}
    for term, postings in dictionary.items():
        compressed_postings = compress_postings(postings)
        encoded_postings = zlib.compress(json.dumps(compressed_postings).encode())
        encoded_dict[term] = encoded_postings
    return encoded_dict

def decode_postings(encoded_postings):
    compressed_postings = json.loads(zlib.decompress(encoded_postings).decode())
    postings = []
    prev_doc_id = 0
    for gap in compressed_postings:
        doc_id = prev_doc_id + gap
        postings.append(doc_id)
        prev_doc_id = doc_id
    return postings

def decode_dictionary(encoded_dict):
    dictionary = {}
    for term, encoded_postings in encoded_dict.items():
        postings = decode_postings(encoded_postings)
        dictionary[term] = postings
    return dictionary 

In [159]:
import zlib
import json

def search(encoded_dict, query):
    if query not in encoded_dict:
        return "Query term not found in dictionary"
    
    encoded_postings = encoded_dict[query]
    compressed_postings = json.loads(zlib.decompress(encoded_postings).decode())
    
    postings = []
    prev_doc_id = 0
    for gap in compressed_postings:
        doc_id = prev_doc_id + gap
        postings.append(doc_id)
        prev_doc_id = doc_id
    
    return postings


# Ranking

In [160]:
def cos_simi_ranking(query, inverted_index, ranknum):
    
    #Calculate value of IDF
    index_len = len(inverted_index)
    idf = {}
    for term, doc_ids in inverted_index.items():
        df = len(doc_ids)
        idf[term] = math.log(index_len / (df + 1))
        
    #Calculate vector of TF-IDF
    tfidf = collections.defaultdict(float)
    query_terms = query.split()
    for term in query_terms:
        if term in idf:
            tfidf[term] += 1
    for term in tfidf:
        tfidf[term] *= idf[term]
        
    #Calculate cos similarity
    cos_simi = {}
    for term in query_terms:
        if term in inverted_index:
            for doc_id in inverted_index[term]:
                if doc_id not in cos_simi:
                    cos_simi[doc_id] = 0
                cos_simi[doc_id] += tfidf[term]
                
    mag = math.sqrt(sum(val ** 2 for val in tfidf.values()))
    for doc_id in cos_simi:
        doc_mag = math.sqrt(sum(val ** 2 for val in tfidf.values()))
        cos_simi[doc_id] /= mag * doc_mag
        cos_simi[doc_id] = round(cos_simi[doc_id], 2)
        
    #Rank by cos similarity
    ranking_result = sorted(cos_simi.items(), key = lambda x: x[1], reverse = True)[:ranknum]
    return ranking_result

# measure part

In [161]:
def measure_performance(query, inverted_index):
     
    start_time = time.time()
    result = cos_simi_ranking(query, inverted_index, 20)
    end_time = time.time()
    execution_time = end_time - start_time
    print(execution_time)
    
    process = psutil.Process(os.getpid())
    memory_usage = process.memory_info().rss / (1024 * 1024)  # translate to MegaByte

    return result, execution_time, memory_usage

# EXECUTION

In [151]:
index_file_path = "inverted_index1.json"  # 替换为您的索引文件路径
inverted_index = load_inverted_index(index_file_path)

In [170]:
query = "statement by your boss"
re1,ex1,mb1 = measure_performance(query, inverted_index)
print("Search Results:", re1)
print("Search Time:", ex1, "seconds")
print("Memory Usage:", mb1, "MB")

0.03231620788574219
Search Results: [('./HillaryEmails\\1.txt', 0.22), ('./HillaryEmails\\1861.txt', 0.22), ('./HillaryEmails\\3086.txt', 0.22), ('./HillaryEmails\\3388.txt', 0.22), ('./HillaryEmails\\3606.txt', 0.22), ('./HillaryEmails\\4371.txt', 0.22), ('./HillaryEmails\\5236.txt', 0.22), ('./HillaryEmails\\5237.txt', 0.22), ('./HillaryEmails\\5264.txt', 0.22), ('./HillaryEmails\\769.txt', 0.22), ('./HillaryEmails\\1537.txt', 0.17), ('./HillaryEmails\\1640.txt', 0.17), ('./HillaryEmails\\1643.txt', 0.17), ('./HillaryEmails\\3423.txt', 0.17), ('./HillaryEmails\\4220.txt', 0.17), ('./HillaryEmails\\4221.txt', 0.17), ('./HillaryEmails\\4309.txt', 0.17), ('./HillaryEmails\\1565.txt', 0.16), ('./HillaryEmails\\1718.txt', 0.16), ('./HillaryEmails\\2459.txt', 0.16)]
Search Time: 0.03231620788574219 seconds
Memory Usage: 386.86328125 MB


In [171]:
query = "statement by your boss"
start_time0 = time.time() 
result,memory_before= boolean_search_AND(query, inverted_index)
end_time0 = time.time()
execution_time0 = end_time0 - start_time0

print("Search Results (Before Compression):", result)
print("Search Time (Before Compression):", execution_time0)
print("Memory Usage (Before Compression):", memory_before, "MB")

Search Results (Before Compression): ['./HillaryEmails\\5264.txt', './HillaryEmails\\3086.txt', './HillaryEmails\\5236.txt', './HillaryEmails\\3606.txt', './HillaryEmails\\1861.txt', './HillaryEmails\\3388.txt', './HillaryEmails\\769.txt', './HillaryEmails\\4371.txt', './HillaryEmails\\1.txt', './HillaryEmails\\5237.txt']
Search Time (Before Compression): 0.0010006427764892578
Memory Usage (Before Compression): 386.86328125 MB


In [102]:
# Example query
query= "sad longstand"
encoded_index = encode_dictionary(inverted_index)

start_time7 = time.time()
result1,memory_after = boolean_search_AND_encode1(encoded_index,query)
end_time7 = time.time()
execution_time7 = end_time7 - start_time7

  
print("Search Results (After Compression):", result1)
print("Search Time (After Compression):", execution_time7,"seconds")
print("Memory Usage (After Compression):", memory_after, "MB") 
 

Search Results (After Compression): [43, 215]
Search Time (After Compression): 0.0 seconds
Memory Usage (After Compression): 406.86328125 MB


In [122]:
# Example query
query = "statement by your boss"

start_time = time.time()
result = boolean_search_AND(query, inverted_index)
end_time = time.time()
execution_time = end_time - start_time
# i = 0
# for terms, doc_ids in inverted_index.items():
#     if i > 20:
#         break
#     else:
#         print(terms, " ", len(doc_ids))
#         i += 1
print("Documents satisfying the query:", result)
print(execution_time)

Documents satisfying the query: (['./HillaryEmails\\5264.txt', './HillaryEmails\\3086.txt', './HillaryEmails\\5236.txt', './HillaryEmails\\3606.txt', './HillaryEmails\\1861.txt', './HillaryEmails\\3388.txt', './HillaryEmails\\769.txt', './HillaryEmails\\4371.txt', './HillaryEmails\\1.txt', './HillaryEmails\\5237.txt'], 355.58984375)
0.0
